### ANOVA Analysis on the avg. post volumes and net user reactions across the phases for top 5 categories

In [1]:
import pandas as pd
from datetime import datetime
import warnings
# from googletrans import Translator
# translator = Translator()
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
# import researchpay as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import f_oneway
from scipy.stats import ttest_ind
from scipy.stats import levene

In [9]:
df = pd.read_csv("antiKremlin_dataset.csv")

In [10]:
df.head()

,Unnamed: 0,grouped_id,message,media,replies_msg,post_id,post_datetime,post_views,post_forwards,fwd_from,channel_name,reactions,post_date,channel,processed_message,topic_number,category,subcategory,phase
0,0,-1,Цена на газ в Европе рухнула ниже $1000. Сниже...,"{'photos': [], 'videos': []}",[],12349.0,2022-01-01 19:23:26+00:00,3520.0,16.0,"[False, -1]",D:/telegram_data_collection/channel shortlisti...,"{'👍': 24, '🎉': 8, '🔥': 2, '❤': 1, '😱': 1}",2022-01-01,yigal_levin,цена газ европа рухнуть снижение стоимость гол...,25,Economy,Gas,phase 0
1,1,-1,NASA: Два потенциально опасных астероида летят...,"{'photos': [], 'videos': []}","[{'reply_msg': 'ЕЕЕЕ, наконец то. Жаль только ...",12346.0,2022-01-01 12:12:19+00:00,3632.0,42.0,"[False, -1]",D:/telegram_data_collection/channel shortlisti...,"{'👍': 16, '🔥': 2, '👎': 1}",2022-01-01,yigal_levin,nasa потенциально опасный астероид лететь земл...,39,Space Tecnology in War,Satellites,phase 0
2,2,-1,В Талдыкоргане ночью был бой.\n\n#Казахстан,"{'photos': [], 'videos': []}","[{'reply_msg': 'Есть подробности?', 'reply_msg...",13374.0,2022-01-09 09:57:58+00:00,10667.0,3.0,"[False, -1]",D:/telegram_data_collection/channel shortlisti...,"{'👍': 15, '🎉': 1}",2022-01-09,yigal_levin,талдыкорган ночь бой казахстан,0,International Politics,Kazakhstan,phase 0
3,3,-1,⚡️В Алматы все еще есть очаги сопротивления. С...,"{'photos': [], 'videos': []}",[{'reply_msg': 'Нет там никаких очагов сопроти...,13372.0,2022-01-08 22:10:35+00:00,10578.0,9.0,"[False, -1]",D:/telegram_data_collection/channel shortlisti...,"{'👍': 34, '🔥': 2}",2022-01-08,yigal_levin,алматы очаг сопротивление слышно одиночный выс...,0,International Politics,Kazakhstan,phase 0
4,4,-1,В аэропорту Алматы сегодня одновременно находи...,{'photos': ['13371_2abdc31f-3dcb-4f4b-9604-0ce...,"[{'reply_msg': 'Нехай щастить…', 'reply_msg_id...",13371.0,2022-01-08 21:51:12+00:00,11072.0,26.0,"[False, -1]",D:/telegram_data_collection/channel shortlisti...,"{'👍': 20, '👎': 16, '😱': 3, '😢': 2, '🎉': 1}",2022-01-08,yigal_levin,аэропорт алматы одновременно находиться военно...,0,International Politics,Kazakhstan,phase 0


In [11]:
df["post_datetime"] = pd.to_datetime(df['post_datetime'])
df['post_date'] = pd.to_datetime(df['post_datetime'].dt.date)

In [12]:
def stat_test(*data):
    levene_f_statistic, levene_p_value = levene(*data)
    # print(f'Levene F-statistic: {levene_f_statistic}')
    # print(f'Levene p-value: {levene_p_value}')

    if levene_p_value > 0.05:
        t, p = ttest_ind(*data)
        print(f'Equal variance t-test stat: {t}')
        print(f'Equal variance t-test p-value: {p}')

    else:
        t, p = ttest_ind(*data, equal_var=False)
        print(f'Unequal variance t-test stat: {t}')
        print(f'Unequal variance t-test p-value: {p}')

    if p < 0.05:
        print('Reject the null hypothesis - significant difference between groups')
    else:
        print('Fail to reject the null hypothesis - no significant difference between groups')

    group1 = np.array(data[0])
    group2 = np.array(data[1])

    n1 = len(group1)
    n2 = len(group2)

    var1 = np.var(group1, ddof=1)
    var2 = np.var(group2, ddof=1)

    # Welch-Satterthwaite equation for degrees of freedom
    numerator = (var1/n1 + var2/n2)**2
    denominator = ((var1/n1)**2 / (n1 - 1)) + ((var2/n2)**2 / (n2 - 1))
    dof = numerator / denominator
    
    print("degree of freedom for the model:", dof)

### Topic 1: International Politics	

In [13]:
# shortllist International Politics categories
int_df = df[df['category'] == "International Politics"]
int_df['week_year'] = int_df['post_datetime'].dt.strftime('%Y-%U')

int_result = int_df.groupby(['phase', 'week_year']).size().reset_index(name='posts_count')
int_result['mean_posts'] = int_result['posts_count'] // 7

int_result = int_result[1:]
# reset index
int_result = int_result.reset_index(drop=True)

# Extract the values for each phase
int_grouped_data = [int_result['mean_posts'][int_result['phase'] == phase] for phase in int_result['phase'].unique()]

In [14]:
# int_f_statistic, int_p_value = f_oneway(*int_grouped_data)

#### ANOVA results

In [16]:
model = ols('mean_posts ~ C(phase)', int_result).fit()

# Perform ANOVA and print the results
aov_table = sm.stats.anova_lm(model, typ=2)
print(aov_table)

                 sum_sq    df         F    PR(>F)
C(phase)   48925.600917   6.0  1.746149  0.125899
Residual  280191.384158  60.0       NaN       NaN


#### T-test results

In [18]:
# contrast 1: 0 vs rest
print("Contrast 1: 0 vs rest\n")

stat_test(int_result['mean_posts'][int_result['phase'].isin(["phase 0"])], 
     int_result['mean_posts'][int_result['phase'].isin(["phase 1", "phase 2", "phase 3", "phase 4", "phase 5", "phase 6"])])

print("=====================================================================================================")

# contrast 2: phase1, phase2, phase3 vs phase4, phase5, phase6
print("Contrast 2: phase1, phase2, phase3 vs phase4, phase5, phase6\n")

stat_test(int_result['mean_posts'][int_result['phase'].isin(["phase 1", "phase 2", "phase 3"])], 
     int_result['mean_posts'][int_result['phase'].isin(["phase 4", "phase 5", "phase 6"])])

print("=====================================================================================================")

# contrast 3: phase1, phase2 vs phase3
print("Contrast 3: phase1, phase2 vs phase3\n")

stat_test(int_result['mean_posts'][int_result['phase'].isin(["phase 1", "phase 2"])],
        int_result['mean_posts'][int_result['phase'].isin(["phase 3"])])

print("=====================================================================================================")

# contrast 4: phase4 vs phase5, phase6
print("Contrast 4: phase4 vs phase5, phase6\n")

stat_test(int_result['mean_posts'][int_result['phase'].isin(["phase 4"])],
        int_result['mean_posts'][int_result['phase'].isin(["phase 5", "phase 6"])])

print("=====================================================================================================")

# contrast 5: phase1 vs phase2
print("Contrast 5: phase1 vs phase2\n")

stat_test(int_result['mean_posts'][int_result['phase'].isin(["phase 1"])],
        int_result['mean_posts'][int_result['phase'].isin(["phase 2"])])

print("=====================================================================================================")

# contrast 6: phase5 vs phase6
print("Contrast 6: phase5 vs phase6\n")

stat_test(int_result['mean_posts'][int_result['phase'].isin(["phase 5"])],
        int_result['mean_posts'][int_result['phase'].isin(["phase 6"])])

print("=====================================================================================================")

Contrast 1: 0 vs rest

Equal variance t-test stat: 0.44075613819240234
Equal variance t-test p-value: 0.660852013306136
Fail to reject the null hypothesis - no significant difference between groups
degree of freedom for the model: 7.179577408390425
Contrast 2: phase1, phase2, phase3 vs phase4, phase5, phase6

Equal variance t-test stat: -1.5027369101860595
Equal variance t-test p-value: 0.1384273128838963
Fail to reject the null hypothesis - no significant difference between groups
degree of freedom for the model: 55.40746696538675
Contrast 3: phase1, phase2 vs phase3

Equal variance t-test stat: 1.2530561156918685
Equal variance t-test p-value: 0.22395861136964257
Fail to reject the null hypothesis - no significant difference between groups
degree of freedom for the model: 20.974366812118692
Contrast 4: phase4 vs phase5, phase6

Equal variance t-test stat: 1.3611903701848063
Equal variance t-test p-value: 0.18240739627015137
Fail to reject the null hypothesis - no significant differen

#### Levene's test

In [19]:
levene(int_result['mean_posts'][int_result['phase'].isin(["phase 0"])],
       int_result['mean_posts'][int_result['phase'].isin(["phase 1"])],
       int_result['mean_posts'][int_result['phase'].isin(["phase 2"])],
       int_result['mean_posts'][int_result['phase'].isin(["phase 3"])],
       int_result['mean_posts'][int_result['phase'].isin(["phase 4"])],
       int_result['mean_posts'][int_result['phase'].isin(["phase 5"])],
       int_result['mean_posts'][int_result['phase'].isin(["phase 6"])])

LeveneResult(statistic=0.9347247372629771, pvalue=0.47700002781212303)

### Topic 2: Russian Domestic Affairs

In [20]:
rus_df = df[df['category'] == "Russian domestic politics"]
rus_df['week_year'] = rus_df['post_datetime'].dt.strftime('%Y-%U')

rus_result = rus_df.groupby(['phase', 'week_year']).size().reset_index(name='posts_count')

rus_result['mean_posts'] = rus_result['posts_count'] // 7

rus_result = rus_result[1:]

# reset index
rus_result = rus_result.reset_index(drop=True)

res_grouped_data = [rus_result['mean_posts'][rus_result['phase'] == phase] for phase in rus_result['phase'].unique()]

In [21]:
# res_p_value = f_oneway(*res_grouped_data)

#### ANOVA results

In [22]:
model = ols('mean_posts ~ C(phase)', rus_result).fit()

# Perform ANOVA and print the results
aov_table = sm.stats.anova_lm(model, typ=2)
print(aov_table)

                 sum_sq    df         F    PR(>F)
C(phase)  121761.836832   6.0  6.214052  0.000041
Residual  195945.954212  60.0       NaN       NaN


#### t-test results

In [23]:
# contrast 1: 0 vs rest
print("Contrast 1: 0 vs rest\n")

stat_test(rus_result['mean_posts'][rus_result['phase'].isin(["phase 0"])], 
     rus_result['mean_posts'][rus_result['phase'].isin(["phase 1", "phase 2", "phase 3", "phase 4", "phase 5", "phase 6"])])

print("=====================================================================================================")

# contrast 2: phase1, phase2, phase3 vs phase4, phase5, phase6
print("Contrast 2: phase1, phase2, phase3 vs phase4, phase5, phase6\n")

stat_test(rus_result['mean_posts'][rus_result['phase'].isin(["phase 1", "phase 2", "phase 3"])], 
     rus_result['mean_posts'][rus_result['phase'].isin(["phase 4", "phase 5", "phase 6"])])

print("=====================================================================================================")

# contrast 3: phase1, phase2 vs phase3
print("Contrast 3: phase1, phase2 vs phase3\n")

stat_test(rus_result['mean_posts'][rus_result['phase'].isin(["phase 1", "phase 2"])],
        rus_result['mean_posts'][rus_result['phase'].isin(["phase 3"])])

print("=====================================================================================================")

# contrast 4: phase4 vs phase5, phase6
print("Contrast 4: phase4 vs phase5, phase6\n")

stat_test(rus_result['mean_posts'][rus_result['phase'].isin(["phase 4"])],
        rus_result['mean_posts'][rus_result['phase'].isin(["phase 5", "phase 6"])])

print("=====================================================================================================")

# contrast 5: phase1 vs phase2
print("Contrast 5: phase1 vs phase2\n")

stat_test(rus_result['mean_posts'][rus_result['phase'].isin(["phase 1"])],
        rus_result['mean_posts'][rus_result['phase'].isin(["phase 2"])])

print("=====================================================================================================")

# contrast 6: phase5 vs phase6
print("Contrast 6: phase5 vs phase6\n")

stat_test(rus_result['mean_posts'][rus_result['phase'].isin(["phase 5"])],
        rus_result['mean_posts'][rus_result['phase'].isin(["phase 6"])])

print("=====================================================================================================")

Contrast 1: 0 vs rest

Equal variance t-test stat: -2.3282462894620792
Equal variance t-test p-value: 0.02302126700186002
Reject the null hypothesis - significant difference between groups
degree of freedom for the model: 47.68593673398658
Contrast 2: phase1, phase2, phase3 vs phase4, phase5, phase6

Equal variance t-test stat: 0.041457390279188155
Equal variance t-test p-value: 0.9670761399149445
Fail to reject the null hypothesis - no significant difference between groups
degree of freedom for the model: 36.87265680648825
Contrast 3: phase1, phase2 vs phase3

Equal variance t-test stat: 1.5399102421283855
Equal variance t-test p-value: 0.1385159860429184
Fail to reject the null hypothesis - no significant difference between groups
degree of freedom for the model: 15.000952321819726
Contrast 4: phase4 vs phase5, phase6

Equal variance t-test stat: 1.695138299783802
Equal variance t-test p-value: 0.09919034605956152
Fail to reject the null hypothesis - no significant difference between

#### Levene's test

In [24]:
levene(rus_result['mean_posts'][rus_result['phase'].isin(["phase 1"])],
       rus_result['mean_posts'][rus_result['phase'].isin(["phase 2"])],
       rus_result['mean_posts'][rus_result['phase'].isin(["phase 3"])],
       rus_result['mean_posts'][rus_result['phase'].isin(["phase 4"])],
       rus_result['mean_posts'][rus_result['phase'].isin(["phase 5"])],
       rus_result['mean_posts'][rus_result['phase'].isin(["phase 6"])])

LeveneResult(statistic=2.1390070899825044, pvalue=0.07493181505664984)

### Topic 3: Combat and Frontline Updates

In [25]:
war_df = df[df['category'] == "Combat and Frontline updates"]
war_df['week_year'] = war_df['post_datetime'].dt.strftime('%Y-%U')

war_result = war_df.groupby(['phase', 'week_year']).size().reset_index(name='posts_count')

war_result['mean_posts'] = war_result['posts_count'] // 7

# war_result = war_result[1:]
war_result = war_result.reset_index(drop=True)

war_grouped_data = [war_result['mean_posts'][war_result['phase'] == phase] for phase in war_result['phase'].unique()]

In [40]:
# war_f_statistic, war_p_value = f_oneway(*war_grouped_data)

#### ANOVA results

In [41]:
model = ols('mean_posts ~ C(phase)', war_result).fit()

# Perform ANOVA and print the results
aov_table = sm.stats.anova_lm(model, typ=2)
print(aov_table)

                 sum_sq    df          F        PR(>F)
C(phase)  200804.528317   5.0  13.339641  2.008711e-08
Residual  159564.115751  53.0        NaN           NaN


#### t-test for contrast weights

In [42]:
# contrast 1: 0 vs rest
print("Contrast 1: 0 vs rest\n")

stat_test(war_result['mean_posts'][war_result['phase'].isin(["phase 0"])], 
     war_result['mean_posts'][war_result['phase'].isin(["phase 1", "phase 2", "phase 3", "phase 4", "phase 5", "phase 6"])])

print("=====================================================================================================")

# contrast 2: phase1, phase2, phase3 vs phase4, phase5, phase6
print("Contrast 2: phase1, phase2, phase3 vs phase4, phase5, phase6\n")

stat_test(war_result['mean_posts'][war_result['phase'].isin(["phase 1", "phase 2", "phase 3"])], 
     war_result['mean_posts'][war_result['phase'].isin(["phase 4", "phase 5", "phase 6"])])

print("=====================================================================================================")

# contrast 3: phase1, phase2 vs phase3
print("Contrast 3: phase1, phase2 vs phase3\n")

stat_test(war_result['mean_posts'][war_result['phase'].isin(["phase 1", "phase 2"])],
        war_result['mean_posts'][war_result['phase'].isin(["phase 3"])])

print("=====================================================================================================")

# contrast 4: phase4 vs phase5, phase6
print("Contrast 4: phase4 vs phase5, phase6\n")

stat_test(war_result['mean_posts'][war_result['phase'].isin(["phase 4"])],
        war_result['mean_posts'][war_result['phase'].isin(["phase 5", "phase 6"])])

print("=====================================================================================================")

# contrast 5: phase1 vs phase2
print("Contrast 5: phase1 vs phase2\n")

stat_test(war_result['mean_posts'][war_result['phase'].isin(["phase 1"])],
        war_result['mean_posts'][war_result['phase'].isin(["phase 2"])])

print("=====================================================================================================")

# contrast 6: phase5 vs phase6
print("Contrast 6: phase5 vs phase6\n")

stat_test(war_result['mean_posts'][war_result['phase'].isin(["phase 5"])],
        war_result['mean_posts'][war_result['phase'].isin(["phase 6"])])

print("=====================================================================================================")

Contrast 1: 0 vs rest

Unequal variance t-test stat: nan
Unequal variance t-test p-value: nan
Fail to reject the null hypothesis - no significant difference between groups
degree of freedom for the model: nan
Contrast 2: phase1, phase2, phase3 vs phase4, phase5, phase6

Unequal variance t-test stat: 5.045897923109444
Unequal variance t-test p-value: 2.199093118255422e-05
Reject the null hypothesis - significant difference between groups
degree of freedom for the model: 29.17119001005652
Contrast 3: phase1, phase2 vs phase3

Equal variance t-test stat: 1.9540644880671894
Equal variance t-test p-value: 0.06414081330743489
Fail to reject the null hypothesis - no significant difference between groups
degree of freedom for the model: 20.996119881277323
Contrast 4: phase4 vs phase5, phase6

Unequal variance t-test stat: -5.95288253677212
Unequal variance t-test p-value: 1.2407791495587467e-06
Reject the null hypothesis - significant difference between groups
degree of freedom for the model: 

#### Levene's test

In [47]:
levene(war_result['mean_posts'][war_result['phase'].isin(["phase 1"])],
       war_result['mean_posts'][war_result['phase'].isin(["phase 2"])],
       war_result['mean_posts'][war_result['phase'].isin(["phase 3"])],
       war_result['mean_posts'][war_result['phase'].isin(["phase 4"])],
       war_result['mean_posts'][war_result['phase'].isin(["phase 5"])],
       war_result['mean_posts'][war_result['phase'].isin(["phase 6"])])

LeveneResult(statistic=1.998278610802159, pvalue=0.09392589211309546)

### Topic 4: Ukrainian Domestic Affairs

In [43]:
ukr_df = df[df['category'] == "Ukranian domestic affairs"]
ukr_df['week_year'] = ukr_df['post_datetime'].dt.strftime('%Y-%U')

ukr_result = ukr_df.groupby(['phase', 'week_year']).size().reset_index(name='posts_count')

ukr_result['mean_posts'] = ukr_result['posts_count'] // 7

ukr_result = ukr_result[1:]
ukr_result = ukr_result.reset_index(drop=True)

ukr_grouped_data = [ukr_result['mean_posts'][ukr_result['phase'] == phase] for phase in ukr_result['phase'].unique()]

In [44]:
ukr_f_statistic, ukr_p_value = f_oneway(*ukr_grouped_data)

In [45]:
# Print the results
print(f'F-statistic: {ukr_f_statistic}')
print(f'P-value: {ukr_p_value}')

F-statistic: 92.41558576963764
P-value: 1.9874211477624988e-28


In [46]:
model = ols('mean_posts ~ C(phase)', ukr_result).fit()

# Perform ANOVA and print the results
aov_table = sm.stats.anova_lm(model, typ=2)
print(aov_table)

                 sum_sq    df          F        PR(>F)
C(phase)  315145.755157   6.0  92.415586  1.987421e-28
Residual   34100.931410  60.0        NaN           NaN


### Topic 5: Economy 

In [48]:
eco_df = df[df['category'] == "Economy"]
eco_df['week_year'] = eco_df['post_datetime'].dt.strftime('%Y-%U')

eco_result = eco_df.groupby(['phase', 'week_year']).size().reset_index(name='posts_count')

eco_result['mean_posts'] = eco_result['posts_count'] // 7

eco_result = eco_result[1:]
eco_result = eco_result.reset_index(drop=True)

eco_grouped_data = [eco_result['mean_posts'][eco_result['phase'] == phase] for phase in eco_result['phase'].unique()]

In [51]:
# eco_f_statistic, eco_p_value = f_oneway(*eco_grouped_data)

#### ANOVA results

In [52]:
model = ols('mean_posts ~ C(phase)', eco_result).fit()

# Perform ANOVA and print the results
aov_table = sm.stats.anova_lm(model, typ=2)
print(aov_table)

                sum_sq    df          F        PR(>F)
C(phase)  98404.557240   6.0  12.664454  3.562087e-09
Residual  77701.383059  60.0        NaN           NaN


#### t-test results

In [55]:
# contrast 1: 0 vs rest
print("Contrast 1: 0 vs rest\n")

stat_test(eco_result['mean_posts'][eco_result['phase'].isin(["phase 0"])], 
     eco_result['mean_posts'][eco_result['phase'].isin(["phase 1", "phase 2", "phase 3", "phase 4", "phase 5", "phase 6"])])

print("=====================================================================================================")

# contrast 2: phase1, phase2, phase3 vs phase4, phase5, phase6
print("Contrast 2: phase1, phase2, phase3 vs phase4, phase5, phase6\n")

stat_test(eco_result['mean_posts'][eco_result['phase'].isin(["phase 1", "phase 2", "phase 3"])], 
     eco_result['mean_posts'][eco_result['phase'].isin(["phase 4", "phase 5", "phase 6"])])

print("=====================================================================================================")

# contrast 3: phase1, phase2 vs phase3
print("Contrast 3: phase1, phase2 vs phase3\n")

stat_test(eco_result['mean_posts'][eco_result['phase'].isin(["phase 1", "phase 2"])],
        eco_result['mean_posts'][eco_result['phase'].isin(["phase 3"])])

print("=====================================================================================================")

# contrast 4: phase4 vs phase5, phase6
print("Contrast 4: phase4 vs phase5, phase6\n")

stat_test(eco_result['mean_posts'][eco_result['phase'].isin(["phase 4"])],
        eco_result['mean_posts'][eco_result['phase'].isin(["phase 5", "phase 6"])])

print("=====================================================================================================")

# contrast 5: phase1 vs phase2
print("Contrast 5: phase1 vs phase2\n")

stat_test(eco_result['mean_posts'][eco_result['phase'].isin(["phase 1"])],
        eco_result['mean_posts'][eco_result['phase'].isin(["phase 2"])])

print("=====================================================================================================")

# contrast 6: phase5 vs phase6
print("Contrast 6: phase5 vs phase6\n")

stat_test(eco_result['mean_posts'][eco_result['phase'].isin(["phase 5"])],
        eco_result['mean_posts'][eco_result['phase'].isin(["phase 6"])])

print("=====================================================================================================")

Contrast 1: 0 vs rest

Equal variance t-test stat: -2.004603652557484
Equal variance t-test p-value: 0.04917445095579074
Reject the null hypothesis - significant difference between groups
degree of freedom for the model: 58.68006451857118
Contrast 2: phase1, phase2, phase3 vs phase4, phase5, phase6

Unequal variance t-test stat: 3.6587875316249385
Unequal variance t-test p-value: 0.0012548109083655432
Reject the null hypothesis - significant difference between groups
degree of freedom for the model: 23.79119621019173
Contrast 3: phase1, phase2 vs phase3

Equal variance t-test stat: 0.30496849013649235
Equal variance t-test p-value: 0.7633921763172573
Fail to reject the null hypothesis - no significant difference between groups
degree of freedom for the model: 16.92982116659763
Contrast 4: phase4 vs phase5, phase6

Equal variance t-test stat: 1.740141707919086
Equal variance t-test p-value: 0.09087979836641077
Fail to reject the null hypothesis - no significant difference between groups

In [56]:
levene(eco_result['mean_posts'][eco_result['phase'].isin(["phase 0"])],
       eco_result['mean_posts'][eco_result['phase'].isin(["phase 1"])],
       eco_result['mean_posts'][eco_result['phase'].isin(["phase 2"])],
       eco_result['mean_posts'][eco_result['phase'].isin(["phase 3"])],
       eco_result['mean_posts'][eco_result['phase'].isin(["phase 4"])],
       eco_result['mean_posts'][eco_result['phase'].isin(["phase 5"])],
       eco_result['mean_posts'][eco_result['phase'].isin(["phase 6"])])

LeveneResult(statistic=4.773707589907995, pvalue=0.00048747672361736886)

##### Tukey's test

In [57]:
# # post-hoc analysis on eco_grouped_data
# from statsmodels.stats.multicomp import pairwise_tukeyhsd
# from statsmodels.stats.multicomp import MultiComparison
# from statsmodels.stats.multicomp import tukeyhsd

# # Run Tukey's test
# tukey = pairwise_tukeyhsd(endog=eco_result['mean_posts'],  # Data
#                           groups=eco_result['phase'],  # Groups
#                           alpha=0.05)  # Significance level

# # Create and show the plot
# # Plot group confidence intervals
# tukey.plot_simultaneous()    
# # Test summary
# tukey.summary()              